In [24]:
import pandas as pd

HIST_PRICE_DF = pd.read_csv (r'/Users/joezhou/Downloads/ALL_Prices.csv', sep='|')

#Create a sample for code testing purposes
# HIST_PRICE_DF = DF[DF['TickName'].isin(['CBA.AX','ANZ.AX','APT.AX','Z1P.AX'])]

#create a baselist for adding on the feature engineered variables
TICKER_LIST = HIST_PRICE_DF['TickName'].drop_duplicates().reset_index(drop=True)

In [25]:
# Freature Engineering at daily level

#Movement relative to Open price
HIST_PRICE_DF['HIGHtakeLOW'] = HIST_PRICE_DF['High']-HIST_PRICE_DF['Low']
HIST_PRICE_DF['CLOSEtakeOPEN'] = HIST_PRICE_DF['Close']-HIST_PRICE_DF['Open']
HIST_PRICE_DF['HIGHdivideLOW'] = HIST_PRICE_DF['High']/HIST_PRICE_DF['Low']
HIST_PRICE_DF['CLOSEdivideOPEN'] = HIST_PRICE_DF['Close']/HIST_PRICE_DF['Open']
HIST_PRICE_DF['absmvmtOPEN']=HIST_PRICE_DF['HIGHtakeLOW'].abs()/HIST_PRICE_DF['Open']



In [26]:
#Feature Engineering at Ticker level

# BASELINE = BASELINE.columns = ['TickerName']
PRICE_MEDIAN = HIST_PRICE_DF.groupby("TickName")["absmvmtOPEN"].median()
PRICE_STD_DEV = HIST_PRICE_DF.groupby("TickName")["absmvmtOPEN"].std()


TICKER_LIST_ADDED = pd.merge(TICKER_LIST,PRICE_MEDIAN,on='TickName').rename(columns={'absmvmtOPEN': 'absmvmtOPEN_MEDIAN'})


TICKER_LIST_ADDED = pd.merge(TICKER_LIST_ADDED,PRICE_STD_DEV,on='TickName').rename(columns={'absmvmtOPEN': 'absmvmtOPEN_STD'})



In [27]:


TICKER_LIST_ADDED.to_excel('/Users/joezhou/Downloads/ALL_Prices_Features.xlsx')  
